In [15]:
import arcpy,os,http.client,json,requests

fgdb = os.getcwd() + os.sep + 'source_data.gdb';

if not arcpy.Exists(fgdb):
   arcpy.CreateFileGDB_management(
       os.path.dirname(fgdb)
      ,os.path.basename(fgdb)
   );

arcpy.Exists(fgdb)

True

In [16]:
def scrape_ags(host,path,fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    headers = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain"};
    conn = http.client.HTTPSConnection(host);
    conn.request("POST",path,"f=json",headers);
    response = conn.getresponse();
    data = response.read();
    json_data = json.loads(data);
    if not 'currentVersion' in json_data:
        raise ValueError("Error, unable to query https://" + host + path);
    extraction_amount = json_data['maxRecordCount'];
    where = "1=1";
    params = "where={}&returnIdsOnly=true&returnGeometry=false&f=json".format(where);
    conn = http.client.HTTPSConnection(host);
    conn.request("POST",path + "/query",params,headers);
    response = conn.getresponse();
    data = response.read();
    json_data = json.loads(data);
    ary_oid   = sorted(json_data['objectIds']);
    oid_name  = json_data['objectIdFieldName'];
    oid_count = len(ary_oid);
    initial_hit = True;
    counter = 0;
    while counter <= oid_count - 1:
        if counter + extraction_amount > oid_count - 1:
            int_max = oid_count - 1;
        else:
            int_max = counter + extraction_amount - 1;
        where = oid_name + ' >= ' + str(ary_oid[counter]) + ' AND ' + oid_name + ' <= ' + str(ary_oid[int_max]);
        print("  pulling records where " + where);
        fields = "*";
        params = "where={}&outFields={}&returnGeometry=true&f=json".format(where, fields);
        conn = http.client.HTTPSConnection(host);
        conn.request("POST",path + "/query",params,headers);
        response = conn.getresponse();
        data = response.read(); 
        json_data = json.loads(data);
        ef = arcpy.AsShape(json_data,True);
        if initial_hit:
            arcpy.management.CopyFeatures(ef,fgdb + os.sep + fc)
            initial_hit = False;
        else:
            arcpy.Append_management(ef,fgdb + os.sep + fc,"NO_TEST");
        counter += extraction_amount;
    conn.close(); 
    del conn;
    print("DONE");
    return True;


In [17]:
host = "services.arcgis.com"; 

path = "/cJ9YHowT8TU7DUyn/ArcGIS/rest/services/CRWU_CREAT_Grid_Historic/FeatureServer/0";
fc   = "CRWU_CREAT_Grid_Historic";
z    = scrape_ags(host,path,fgdb,fc);


  pulling records where OBJECTID >= 1 AND OBJECTID <= 2000
  pulling records where OBJECTID >= 2001 AND OBJECTID <= 4000
  pulling records where OBJECTID >= 4001 AND OBJECTID <= 6000
  pulling records where OBJECTID >= 6001 AND OBJECTID <= 8000
  pulling records where OBJECTID >= 8001 AND OBJECTID <= 10000
  pulling records where OBJECTID >= 10001 AND OBJECTID <= 12000
  pulling records where OBJECTID >= 12001 AND OBJECTID <= 14000
  pulling records where OBJECTID >= 14001 AND OBJECTID <= 16000
  pulling records where OBJECTID >= 16001 AND OBJECTID <= 18000
  pulling records where OBJECTID >= 18001 AND OBJECTID <= 20000
  pulling records where OBJECTID >= 20001 AND OBJECTID <= 22000
  pulling records where OBJECTID >= 22001 AND OBJECTID <= 24000
  pulling records where OBJECTID >= 24001 AND OBJECTID <= 24743
DONE


In [18]:
path = "/cJ9YHowT8TU7DUyn/ArcGIS/rest/services/CRWU_CREAT_Historic_Climate_Stations/FeatureServer/0";
fc   = "CRWU_CREAT_Historic_Climate_Stations";
z    = scrape_ags(host,path,fgdb,fc);


  pulling records where OBJECTID >= 1 AND OBJECTID <= 2000
  pulling records where OBJECTID >= 2001 AND OBJECTID <= 4000
  pulling records where OBJECTID >= 4001 AND OBJECTID <= 6000
  pulling records where OBJECTID >= 6001 AND OBJECTID <= 8000
  pulling records where OBJECTID >= 8001 AND OBJECTID <= 10000
  pulling records where OBJECTID >= 10001 AND OBJECTID <= 11165
DONE


In [19]:
path = "/cJ9YHowT8TU7DUyn/ArcGIS/rest/services/CRWU_CREAT_Grid_Projections/FeatureServer/0";
fc   = "CRWU_CREAT_Grid_Projections";
z    = scrape_ags(host,path,fgdb,fc);


  pulling records where OBJECTID >= 1 AND OBJECTID <= 2000
  pulling records where OBJECTID >= 2001 AND OBJECTID <= 4000
  pulling records where OBJECTID >= 4001 AND OBJECTID <= 6000
  pulling records where OBJECTID >= 6001 AND OBJECTID <= 8000
  pulling records where OBJECTID >= 8001 AND OBJECTID <= 10000
  pulling records where OBJECTID >= 10001 AND OBJECTID <= 12000
  pulling records where OBJECTID >= 12001 AND OBJECTID <= 14000
  pulling records where OBJECTID >= 14001 AND OBJECTID <= 16000
  pulling records where OBJECTID >= 16001 AND OBJECTID <= 18000
  pulling records where OBJECTID >= 18001 AND OBJECTID <= 20000
  pulling records where OBJECTID >= 20001 AND OBJECTID <= 22000
  pulling records where OBJECTID >= 22001 AND OBJECTID <= 24000
  pulling records where OBJECTID >= 24001 AND OBJECTID <= 24743
DONE


In [20]:
def csv2fc(url,fgdb,fc):
    if arcpy.Exists('tempTable.csv'):
        arcpy.Delete_management('tempTable.csv');
    if arcpy.Exists(fgdb + '/tempTable'):
        arcpy.Delete_management(fgdb + '/tempTable');
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    print("  downloading csv");
    with open('tempTable.csv','wb') as f,requests.get(url,stream=True) as r:
        for line in r.iter_lines():
            f.write(line + '\n'.encode());
    print("  loading to table");
    arcpy.TableToTable_conversion('tempTable.csv',fgdb,'tempTable')
    print("  converting to points");
    arcpy.management.XYTableToPoint(
        fgdb + os.sep + 'tempTable'
        ,fgdb + os.sep + fc
        ,'longitude'
        ,'latitude'
    );
    arcpy.Delete_management(fgdb + '/tempTable');
    arcpy.Delete_management('tempTable.csv');
    print("DONE");
    return True;


In [21]:
url = "https://raw.githubusercontent.com/barrc/get_ncei/master/src/coop_stations_to_use.csv"
z   = csv2fc(url,fgdb,'COOP_STATIONS_TO_USE');


  downloading csv
  loading to table
  converting to points
DONE


In [22]:
url = "https://raw.githubusercontent.com/barrc/get_ncei/master/src/isd_stations_to_use.csv"
z   = csv2fc(url,fgdb,'ISD_STATIONS_TO_USE');

  downloading csv
  loading to table
  converting to points
DONE
